In [65]:
import numpy as np
import time
from math import inf
import heapq as hq

In [77]:
class vertex_maker:
    def __init__(self, node):
        self.id = node
        self.adjacent = {}
        self.adjacent_R = {}
        self.distance = inf
        self.distance_R = inf
        self.visited = False
        slef.visited_R = False
        self.prev = None
        self.prev_R = None
    
    def add_neighbor(self, neighbor, weight=0):
        self.adjacent[neighbor] = weight
 
    def get_weight(self, neighbor):
        return self.adjacent[neighbor]
    
    # For reversed neighbor vertex
    def add_neighbor_R(self, neighbor, weight=0):
        self.adjacent_R[neighbor] = weight
    
    def get_weight_R(self, neighbor):
        return self.adjacent_R[neighbor]

    # 대소비교를 위함
    def __lt__(self, other):
#         self.distance_f < other.distance_f
        self.distance_R < other.distance_R
    
    
class Graph_maker:
    def __init__(self):
        self.vert_dict = {}
        
    def add_vertex(self, node):
        new_node = vertex_maker(node)
        self.vert_dict[node] = new_node
        
    def add_edges(self, frm, to, cost=0):
        if frm not in self.vert_dict:
            self.add_vertex(frm)
        if to not in self.vert_dict:
            self.add_vertex(to)
        self.vert_dict[frm].add_neighbor(self.vert_dict[to], cost)
        # to make reversed graph
        self.vert_dict[to].add_neighbor_R(self.vert_dict[frm], cost)
        

# Testing questionable graph
g = Graph_maker()

# add vertex
g.add_vertex('a')
g.add_vertex('b')
g.add_vertex('c')
g.add_vertex('d')
g.add_vertex('e')

# add cost btw vertex
g.add_edges('a', 'b', 1)
g.add_edges('b', 'c', 6)
g.add_edges('b', 'd', 8)
g.add_edges('a', 'b', 1)
g.add_edges('c', 'd', 7)
g.add_edges('d', 'e', 1)

# new graph
g2 = Graph_maker()

# add vertex
g2.add_vertex('a')
g2.add_vertex('b')
g2.add_vertex('c')
g2.add_vertex('d')
g2.add_vertex('e')
g2.add_vertex('f')
g2.add_vertex('g')
g2.add_vertex('h')
g2.add_vertex('i')

# add cost btw vertex
g2.add_edges('a', 'b', 4)  
g2.add_edges('a', 'c', 8)
g2.add_edges('b', 'c', 11)
g2.add_edges('b', 'd', 8)
g2.add_edges('c', 'e', 7)
g2.add_edges('c', 'f', 1)
g2.add_edges('d', 'e', 2)
g2.add_edges('d', 'g', 7)
g2.add_edges('d', 'h', 4)
g2.add_edges('e', 'f', 6)
g2.add_edges('f', 'h', 2)
g2.add_edges('g', 'h', 14)
g2.add_edges('g', 'i', 9)
g2.add_edges('h', 'i', 10)

In [123]:
# Bidirectional Dijkstra
def bi_dijkstra(g, start, target):
    # Initialize for implementation
    # open_lst is with priority queue
    open_lst = []
    open_lst_R = []
    closed_lst =[]
    closed_lst_R = []
    hq.heapify(open_lst)
    hq.heapify(open_lst_R)
    
    for i in g.vert_dict.values():
        i.distance = inf
        i.prev = None
        i.distance_R = inf
        i.prev_R = None
        i.visited = False
        i.visited_R = False
    
    start = g.vert_dict[start]
    target = g.vert_dict[target]

    start_time = time.time()

    state = 1
    
#   Initialize start & target node
    start.distance = 0
    start.prev = -1
    target.distance_R = 0
    target.prev_R = -1
    
    closed_lst.append(start)
    closed_lst_R.append(target)
    
    current = start
    current_R = target

    while True:
        # forward search
        state += 1
        for near_vtx, near_vtx_cost in current.adjacent.items():
            if near_vtx.visited == True: continue
            if near_vtx.distance > current.distance + near_vtx_cost:
                near_vtx.distance = current.distance + near_vtx_cost
                near_vtx.prev = current
                hq.heappush(open_lst, (near_vtx.distance, near_vtx))        

        # By heappop current node is deleted from open list
        nxt_node = hq.heappop(open_lst)
        # Choose the next node
        current = nxt_node[1]
        
        current.visited = True
        closed_lst.append(current)
        if current.visited_R == True: break
        
        # backward search
        state += 1
        for near_vtx, near_vtx_cost in current_R.adjacent_R.items():
            if near_vtx.visited_R == True: continue
            if near_vtx.distance_R > current_R.distance_R + near_vtx_cost:
                near_vtx.distance_R = current_R.distance_R + near_vtx_cost
                near_vtx.prev_R = current_R
                hq.heappush(open_lst_R, (near_vtx.distance_R, near_vtx))        

        # By heappop current node is deleted from open list
        nxt_node = hq.heappop(open_lst_R)
        # Choose the next node
        current_R = nxt_node[1]
        
        current_R.visited_R = True
        closed_lst_R.append(current_R)
        if current_R.visited == True: break
    
    # End Phase
    while True:    
        # forward search
        state += 1
        for near_vtx, near_vtx_cost in current.adjacent.items():
            if near_vtx.visited == True: continue
            if near_vtx.distance > current.distance + near_vtx_cost:
                near_vtx.distance = current.distance + near_vtx_cost
                near_vtx.prev = current

        # By heappop current node is deleted from open list
        nxt_node = hq.heappop(open_lst)
        # Choose the next node
        current = nxt_node[1]
        closed_lst.append(current)
        if len(open_lst) == 0 & len(open_lst_R) == 0: break

        # backward search
        state += 1
        for near_vtx, near_vtx_cost in current_R.adjacent_R.items():
            if near_vtx.visited_R == True: continue
            if near_vtx.distance_R > current_R.distance_R + near_vtx_cost:
                near_vtx.distance_R = current_R.distance_R + near_vtx_cost
                near_vtx.prev_R = current_R

        # By heappop current node is deleted from open list
        nxt_node = hq.heappop(open_lst_R)
        # Choose the next node
        current_R = nxt_node[1]
        closed_lst_R.append(current_R)
        if len(open_lst) == 0 & len(open_lst_R) == 0: break
    
    
    z_lst = list(set(closed_lst)&set(closed_lst_R))
    node_lst = []
    for i in z_lst:
        node_lst.append((i.distance+i.distance_R, i))
    
    node = min(node_lst)[1]
            
    execution_time = time.time() - start_time
    
    # shortest forward path
    path_node = node
    path_f = [path_node.id]    
    while path_node != start:
        path_node = path_node.prev
        path_f.append(path_node.id)
    path_f.reverse()
    
    # shortest backward path
    path_node = node
    path_R = [path_node.id]
    while path_node != target:
        path_node = path_node.prev_R
        path_R.append(path_node.id)

    path = path_f+path_R[1:]
    
    print("[Bidirectional Dijkstra]")
    print(">>> Shortest Path from {} to {} is:".format(start.id, target.id), path)
    print(">>> distance cost is:", node.distance + node.distance_R)
    print(">>> #iteration: ", state)
    print(">>> execution time: ",execution_time)
    
    return path

In [127]:
bi_dijkstra(g, 'a', 'e')

[Bidirectional Dijkstra]
>>> Shortest Path from a to e is: ['a', 'b', 'd', 'e']
>>> distance cost is: 10
>>> #iteration:  6
>>> execution time:  1.3828277587890625e-05


['a', 'b', 'd', 'e']

In [132]:
bi_dijkstra(g2, 'a', 'i')

[Bidirectional Dijkstra]
>>> Shortest Path from a to i is: ['a', 'c', 'f', 'h', 'i']
>>> distance cost is: 21
>>> #iteration:  10
>>> execution time:  2.193450927734375e-05


['a', 'c', 'f', 'h', 'i']